Programa que separa a las observaciones en distintos conjuntos según su S/N,
y que calcula los TOAs para cada una de las observaciones dentro de esas categorías

In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import pypulse as pulsar
import matplotlib.pyplot as plt     # para graficar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import os
import shutil

In [2]:
ant = 'A1'                                         # antena usada (A1 o A2)    
template = "template_" + ant + ".std"  # nombre del template
obs_template = np.loadtxt( './templates/obs_template_' + ant + '.dat' , skiprows=0 , dtype=str , usecols=(0) )

In [3]:
#creamos las carpetas que contendrán los resultados

dirName = "./results_sn_" + ant          # nombre de la carpeta en que guardaremos los resultados
if not os.path.exists(dirName):
    os.mkdir(dirName)                         # si la carpeta no existe desde antes, la creamos
    
os.chdir(dirName + '/')                       # entramos en la carpeta en la cual guadaremos los resultados


if not os.path.exists('./lists'):
    os.mkdir('./lists')                         # creamos la carpeta que contendrá las listas
    
    
if not os.path.exists('./timing_files'):
    os.mkdir('./timing_files')                         # creamos la carpeta que contendrá los archivos .tim          
    
os.chdir('..')                                # salimos de la carpeta en la cual guadaremos los resultados

# copiamos también el archivo .par a las nueva carpeta,
# ya que después lo necesitaremos para usar tempo2

shutil.copy('./J0437-4715.par', "./results_sn_" + ant + "/timing_files")


# además, copiamos los tempaltes a las carpetas que contienen los .pfd,
# ya que los necesitaremos luego para poder calcular TOAs

shutil.copy('./templates/' + template, './' + ant + '/')             # copiamos el template a la carpeta

'./A1/template_A1.std'

In [ ]:
#----------------------------------------------------------------------------
# Cargamos las observaciones a Pypulse
#----------------------------------------------------------------------------           

# creamos una lista con todos los archivos .pfd de observaciones

os.chdir('./' + ant + '/')                         # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*pfd')                           # lista de archivos .pfd

# Convert PFD files to PSRFITS

for pfd in pfd_files:
    subprocess.check_call(['psrconv','-o','PSRFITS','-e','fits',pfd])
    
# Save all PSRFITS files
psrfits_files = glob.glob('*fits')

# Load all PSRFITS into PyPulse
fits_pypulse= []
for psrfits in psrfits_files:
    temp_pypulse= pulsar.Archive(psrfits)
    fits_pypulse.append(temp_pypulse)

observations = dict(zip(psrfits_files, fits_pypulse))

# Create single pulse object
# Creat array of best profiles

single_pulses=[]

for observation in fits_pypulse:
    
    # First crunch in time and frequency
    observation.fscrunch()
    observation.tscrunch()
    
    # Now get array of the best profile
    tmp_singlepulse = observation.getSinglePulses(windowsize=256)
    
    # Align and normalize
    tmp_singlepulse.center_align()
    tmp_singlepulse.normalize()
    
    # Save aligned and normalize in each observation
    single_pulses.append(tmp_singlepulse)
    
# Save single pulse object
sp_observations = dict(zip(psrfits_files, single_pulses))

os.chdir('..')                                 # salimos del directorio que contiene los .pfds

Loading: prepfold_20190520_115652_PSR_0437-4715.fits
Load time: 1.62 s
Loading: ds4096_J0437-4715_A1_20190927_040901_0.00-0.85_PSR_0437-4715.fits
Load time: 1.65 s
Loading: prepfold_20190621_094647_PSR_0437-4715.fits
Load time: 0.80 s
Loading: prepfold_20190903_045245_PSR_0437-4715.fits
Load time: 0.79 s
Loading: prepfold_20190728_073037_PSR_0437-4715.fits
Load time: 0.79 s
Loading: prepfold_20190929_030911_PSR_0437-4715.fits
Load time: 0.79 s
Loading: ds4096_J0437-4715_A1_20190922_041010_0.00-0.90_PSR_0437-4715.fits
Load time: 1.65 s
Loading: ds4096_J0437-4715_A1_20190912_042357_0.00-0.86_PSR_0437-4715.fits
Load time: 1.68 s
Loading: prepfold_20190608_112623_PSR_0437-4715.fits
Load time: 1.57 s
Loading: prepfold_20190623_103438_PSR_0437-4715.fits
Load time: 0.80 s
Loading: prepfold_20190703_094122_PSR_0437-4715.fits
Load time: 0.80 s
Loading: prepfold_20190930_030516_PSR_0437-4715.fits
Load time: 0.79 s
Loading: prepfold_20190610_103229_PSR_0437-4715.fits
Load time: 0.80 s
Loading: pr

In [28]:
#----------------------------------------------------------
# Ahora trabajamos con los .pfds de S/N>1,50,80,110,140,170
#----------------------------------------------------------

for y in [1, 50, 80, 110, 140, 170]:
    
    if y == 1:
        sn = 'total'
    else:
        sn = str(y)
        
    files = []                                                                   # listado de observaciones
    list = open("./results_sn_" + ant + "/lists/list_" + str(sn) + ".txt", "w+") # archivo que contendrá S/N y t_obs
    
    for key in observations:
        
        time = (observations[key].getDuration())/60                        # calculamos el tiempo de integración en minutos
        sn_obs = sp_observations[key].getSN()                              # calculamos el S/N de la observación
        file = key.replace(".fits", ".pfd")                                # arreglamos el nombre del archivo para que en lugar de .fits diga .pfd
    
        if sn_obs > y:                                                     # si el S/N es mayor que el valor pedido
            files.append(file)                                             # la agregamos a la lista
            list.write(file + "   " + str(sn_obs) + "   " + str(time) + "\n")  # y escribimos el S/N y el t_obs en el archivo de salida
        
    list.close()                                                       # cerramos el archivo de salida

#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

# primero, generamos un string que contenga todos los archivos .pfd

    files.remove(str(obs_template))     # sacamos de la lista de observaciones a la que usamos como template

    archivos = ''
    for pfd in files:
        archivos = archivos + pfd + ' '
        
# ahora llamamos a pat para generar el archivo .tim

    timing_file = "timing_" + str(sn) + ".tim"

    os.chdir('./' + ant + '/')                  # entramos al directorio que contiene los .pfds
    
    subprocess.call("pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, shell = True)
    
    os.chdir('..')                              # saliemos del directorio que contiene los .pfds
    
# si las observaciones son de A1, hay que hacer una corrección por la frecuencia

    if ant == 'A1':
        subprocess.call(["sed -i -e 's/1414.8750/1414.4375/g' ./results_sn_" + ant + "/timing_files/" + timing_file], shell=True)

'./results_sn_A1/timing_files/J0437-4715.par'